In [17]:
import pandas as pd
import sys
import csv

In [18]:
results = pd.read_csv("Updated_RQ_results.csv")

In [19]:
results

,Unnamed: 0,project,method,algorithm,batch_size,builds_reqd,batch_delays,sbs_delays
0,759.0,cloud_controller_ng.csv,gr,BATCHBISECT,2,76.284585,127.0,0.0
1,778.0,cloud_controller_ng.csv,gr,BATCHBISECT,4,76.679842,381.0,0.0
2,797.0,cloud_controller_ng.csv,gr,BATCH4,4,75.889328,381.0,0.0
3,816.0,cloud_controller_ng.csv,gr,BATCHSTOP4,4,75.889328,381.0,0.0
4,835.0,cloud_controller_ng.csv,gr,BATCHBISECT,8,82.213439,893.0,0.0
...,...,...,...,...,...,...,...,...
595,147.0,gradle.csv,ml,BATCHSTOP4,16,29.802513,4080.0,1.0
596,148.0,gradle.csv,ml,BATCHBISECT,2,35.547576,174.0,24.0
597,149.0,gradle.csv,ml,BATCHBISECT,4,31.956912,660.0,8.0
598,150.0,gradle.csv,ml,BATCHBISECT,8,32.495512,1820.0,0.0


In [20]:
results = results.drop('Unnamed: 0', axis='columns')

In [21]:
batch_delays = results['batch_delays'].tolist()
sbs_delays = results['sbs_delays'].tolist()
total_delays = []

In [22]:
for count in range(len(batch_delays)):
    total_delays.append( batch_delays[count] + sbs_delays[count])

In [23]:
results['total_delay'] = total_delays

In [24]:
results

,project,method,algorithm,batch_size,builds_reqd,batch_delays,sbs_delays,total_delay
0,cloud_controller_ng.csv,gr,BATCHBISECT,2,76.284585,127.0,0.0,127.0
1,cloud_controller_ng.csv,gr,BATCHBISECT,4,76.679842,381.0,0.0,381.0
2,cloud_controller_ng.csv,gr,BATCH4,4,75.889328,381.0,0.0,381.0
3,cloud_controller_ng.csv,gr,BATCHSTOP4,4,75.889328,381.0,0.0,381.0
4,cloud_controller_ng.csv,gr,BATCHBISECT,8,82.213439,893.0,0.0,893.0
...,...,...,...,...,...,...,...,...
595,gradle.csv,ml,BATCHSTOP4,16,29.802513,4080.0,1.0,4081.0
596,gradle.csv,ml,BATCHBISECT,2,35.547576,174.0,24.0,198.0
597,gradle.csv,ml,BATCHBISECT,4,31.956912,660.0,8.0,668.0
598,gradle.csv,ml,BATCHBISECT,8,32.495512,1820.0,0.0,1820.0


In [25]:
method = ['ml', 'ssr', 'gr']
algorithms = ['NOBATCH', 'BATCH4', 'BATCHBISECT', 'BATCHSTOP4']
batchsize = [2, 4, 8, 16]
project_list = list(set(results['project'].tolist()))

In [26]:
for m in method:
    
    filename = m + '_pivot_result_old.csv'
    file = open(filename, 'w')
    res_headers = ['project', 'NB_builds', 'NB_delay', 'NB_Batch', 'B4_builds', 'B4_delay', 'B4_Batch', 'BB_builds', 'BB_delay', 'BB_Batch', 'BS4_builds', 'BS4_delay', 'BS4_Batch']
    res_writer = csv.writer(file)
    res_writer.writerow(res_headers)
    
    
    for p in project_list:
        
        nb_builds = 0
        nb_delay = 0
        nb_batch = 0
        
        b4_builds = 0
        b4_delay = 0
        b4_batch = 0
        
        bs4_builds = 0
        bs4_delay = 0
        bs4_batch = 0
        
        bb_builds = 0
        bb_delay = 0
        bb_batch = 0
        
        for alg in algorithms:
            data = results[ (results['method'] == m) & (results['algorithm'] == alg) & (results['project'] == p)]
            data = data.drop(data[data.batch_size < 2].index)
            data = data.drop(data[data.batch_size > 16].index)

            best_batch = 0
            lowest_distance = sys.maxsize
            delays = data['total_delay'].tolist()
            builds_reqd = data['builds_reqd'].tolist()
            batch_size = data['batch_size'].tolist()

            for i in range(len(delays)):
                d = (builds_reqd[i]**2 + delays[i]**2)**0.5
                if d < lowest_distance:
                    
                    best_batch = batch_size[i]
                    lowest_distance = d
                    
                    if alg == 'NOBATCH':
                        nb_builds = builds_reqd[i]
                        nb_delay = delays[i]
                        nb_batch = best_batch
                    elif alg == 'BATCHBISECT':
                        bb_builds = builds_reqd[i]
                        bb_delays = delays[i]
                        bb_batch = best_batch
                    elif alg == 'BATCHSTOP4':
                        bs4_builds = builds_reqd[i]
                        bs4_delays = delays[i]
                        bs4_batch = best_batch
                    else:
                        b4_builds = builds_reqd[i]
                        b4_delays = delays[i]
                        b4_batch = best_batch
            print('{} - {} - {} - {}'.format(p, m, alg, best_batch))
        res_writer.writerow([p, nb_builds, nb_delay, nb_batch, b4_builds, b4_delays, b4_batch, bb_builds, bb_delays, bb_batch, bs4_builds, bs4_delays, bs4_batch])
        

gradle.csv - ml - NOBATCH - 0
gradle.csv - ml - BATCH4 - 4
gradle.csv - ml - BATCHBISECT - 2
gradle.csv - ml - BATCHSTOP4 - 4
open-build-service.csv - ml - NOBATCH - 0
open-build-service.csv - ml - BATCH4 - 4
open-build-service.csv - ml - BATCHBISECT - 2
open-build-service.csv - ml - BATCHSTOP4 - 4
puppet.csv - ml - NOBATCH - 0
puppet.csv - ml - BATCH4 - 4
puppet.csv - ml - BATCHBISECT - 2
puppet.csv - ml - BATCHSTOP4 - 4
sufia.csv - ml - NOBATCH - 0
sufia.csv - ml - BATCH4 - 4
sufia.csv - ml - BATCHBISECT - 4
sufia.csv - ml - BATCHSTOP4 - 4
loomio.csv - ml - NOBATCH - 0
loomio.csv - ml - BATCH4 - 4
loomio.csv - ml - BATCHBISECT - 2
loomio.csv - ml - BATCHSTOP4 - 4
geoserver.csv - ml - NOBATCH - 0
geoserver.csv - ml - BATCH4 - 4
geoserver.csv - ml - BATCHBISECT - 2
geoserver.csv - ml - BATCHSTOP4 - 4
metasploit-framework.csv - ml - NOBATCH - 0
metasploit-framework.csv - ml - BATCH4 - 4
metasploit-framework.csv - ml - BATCHBISECT - 2
metasploit-framework.csv - ml - BATCHSTOP4 - 4
concer

In [27]:
batch_reps = {0: 'na', 2: '^*^*', 4: '^\#', 6: '^\%', 8:'^\#^\#', 16:'^\wedge'}

In [28]:
print(batch_reps[2])

^*^*


In [29]:
ssr = pd.read_csv('ssr_pivot_result.csv')

In [30]:
projects = ssr['project'].tolist()
col1 = ssr['NB_builds'].tolist()
col2 = ssr['NB_delay'].tolist()
nb_batch = ssr['NB_Batch'].tolist()
col3 = ssr['B4_builds'].tolist()
col4 = ssr['B4_delay'].tolist()
b4_batch = ssr['B4_Batch'].tolist()
col5 = ssr['BB_builds'].tolist()
col6 = ssr['BB_delay'].tolist()
bb_batch = ssr['BB_Batch'].tolist()
col7 = ssr['BS4_builds'].tolist()
col8 = ssr['BS4_delay'].tolist()
bs4_batch = ssr['BS4_Batch'].tolist()

In [15]:
print(batch_reps[nb_batch[3]])

na


In [16]:
for i in range(len(projects)):
    print("{} & {}{} & {}{} & {}{} & {}{} & {}{} & {}{} \\\\".format(projects[i],  
                                                                round(col3[i], 3), batch_reps[b4_batch[i]], round(col4[i], 3), batch_reps[b4_batch[i]], 
                                                                round(col5[i], 3), batch_reps[bb_batch[i]], round(col6[i], 3), batch_reps[bb_batch[i]], 
                                                                round(col7[i], 3), batch_reps[bs4_batch[i]], round(col8[i], 3), batch_reps[bs4_batch[i]] ))
    print("\hline")

gradle.csv & 8.259^\# & 318.0^\# & 15.619^*^* & 138.0^*^* & 8.259^\# & 318.0^\# \\
\hline
open-build-service.csv & 29.614^\# & 474.0^\# & 41.631^*^* & 182.0^*^* & 29.614^\# & 474.0^\# \\
\hline
puppet.csv & 4.348^\# & 24.0^\# & 4.348^*^* & 9.0^*^* & 4.348^\# & 24.0^\# \\
\hline
sufia.csv & 0.0^\# & 6.0^\# & 0.0^*^* & 6.0^*^* & 0.0^\# & 6.0^\# \\
\hline
loomio.csv & 43.902^\# & 211.0^\# & 41.463^*^* & 81.0^*^* & 43.902^\# & 211.0^\# \\
\hline
geoserver.csv & 74.405^\# & 241.0^\# & 70.238^*^* & 93.0^*^* & 74.405^\# & 241.0^\# \\
\hline
metasploit-framework.csv & 8.171^\# & 394.0^\# & 16.732^*^* & 173.0^*^* & 8.171^\# & 394.0^\# \\
\hline
concerto.csv & 13.103^\# & 209.0^\# & 30.69^*^* & 74.0^*^* & 13.103^\# & 209.0^\# \\
\hline
cloud_controller_ng.csv & 35.573^\# & 287.0^\# & 42.292^*^* & 113.0^*^* & 35.573^\# & 287.0^\# \\
\hline
onarqube.csv & 0.0na & 287.0na & 29.0^\#^\# & 624.0^\#^\# & 0.0na & 287.0na \\
\hline
fog.csv & 48.148^\# & 363.0^\# & 49.573^*^* & 144.0^*^* & 48.148^\# & 363

In [108]:
ml = pd.read_csv('ml_pivot_result.csv')

In [112]:
projects = ml['project'].tolist()
col1 = ml['NB_builds'].tolist()
col2 = ml['NB_delay'].tolist()
nb_batch = ml['NB_Batch'].tolist()
col3 = ml['B4_builds'].tolist()
col4 = ml['B4_delay'].tolist()
b4_batch = ml['B4_Batch'].tolist()
col5 = ml['BB_builds'].tolist()
col6 = ml['BB_delay'].tolist()
bb_batch = ml['BB_Batch'].tolist()
col7 = ml['BS4_builds'].tolist()
col8 = ml['BS4_delay'].tolist()
bs4_batch = ml['BS4_Batch'].tolist()

In [114]:
for i in range(len(projects)):
    print("{} & {}{} & {}{} & {}{} & {}{} & {}{} & {}{} & {}{} & {}{} \\\\".format(projects[i], 
                                                                round(col1[i], 3), batch_reps[nb_batch[i]], round(col2[i], 3), batch_reps[nb_batch[i]], 
                                                                round(col3[i], 3), batch_reps[b4_batch[i]], round(col4[i], 3), batch_reps[b4_batch[i]], 
                                                                round(col5[i], 3), batch_reps[bb_batch[i]], round(col6[i], 3), batch_reps[bb_batch[i]], 
                                                                round(col7[i], 3), batch_reps[bs4_batch[i]], round(col8[i], 3), batch_reps[bs4_batch[i]] ))
    print("\hline")

rails.csv & 43.399^*^* & 641.0^*^* & 58.158^\# & 2071.0^\# & 59.377^*^* & 651.0^*^* & 58.158^\# & 2071.0^\# \\
\hline
jruby.csv & 45.036^*^* & 186.0^*^* & 77.724^\# & 588.0^\# & 70.944^*^* & 194.0^*^* & 77.724^\# & 588.0^\# \\
\hline
cloud_controller_ng.csv & 42.688^*^* & 108.0^*^* & 65.217^\# & 351.0^\# & 57.312^*^* & 119.0^*^* & 65.217^\# & 351.0^\# \\
\hline
geoserver.csv & 44.643^*^* & 78.0^*^* & 98.81^\# & 252.0^\# & 83.929^*^* & 79.0^*^* & 98.81^\# & 252.0^\# \\
\hline
orbeon-forms.csv & 45.494^*^* & 106.0^*^* & 54.936^\# & 336.0^\# & 62.661^*^* & 106.0^*^* & 54.936^\# & 336.0^\# \\
\hline
sufia.csv & 26.316^\# & 30.0^\# & 47.368^\# & 30.0^\# & 52.632^*^* & 8.0^*^* & 47.368^\# & 30.0^\# \\
\hline
loomio.csv & 44.39^*^* & 91.0^*^* & 69.268^\# & 300.0^\# & 68.293^*^* & 97.0^*^* & 69.268^\# & 300.0^\# \\
\hline
puppet.csv & 37.681^*^* & 26.0^*^* & 26.087^\# & 86.0^\# & 39.13^*^* & 27.0^*^* & 26.087^\# & 86.0^\# \\
\hline
fog.csv & 44.444^*^* & 158.0^*^* & 67.236^\# & 507.0^\# & 68.3